https://www.youtube.com/c/HuggingFace/playlists

In [ ]:
%pip install transformers

Import necessary libraries

In [ ]:
import io
import os
import torch
from sklearn import metrics
import pandas as pd
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# from ml_things import plot_dict, plot_confusion_matrix, fix_text
from transformers import (set_seed,TrainingArguments,Trainer,AdamW,get_linear_schedule_with_warmup)
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
df_sentiment = pd.read_csv('IMDB6000.csv')

A torch.device is an object representing the device on which a torch.Tensor is or will be allocated.

https://pytorch.org/docs/stable/tensor_attributes.html


In [ ]:
epochs = 2
batch_size = 16
max_length = 128
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_labels = 3

Tokenizing is used to transform raw text into numbers. The three main types of tokenizers used in Transformers: Byte-Pair Encoding (BPE), WordPiece, and SentencePiece

https://huggingface.co/docs/transformers/tokenizer_summary

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased',return_dict=True,output_hidden_states=True)
        self.l2 = torch.nn.Linear(768,n_labels)
    
    def forward(self, ids, mask, token_type_ids):
        output_1= self.l1(ids, attention_mask = mask)
        output_1 = output_1.hidden_states[-1][:,0,:]
        output_2 = self.l2(output_1)
        return output_2

model = BERTClass()
model = model.to(device)

In [ ]:
df_sentiment.head()

,Unnamed: 0,review,sentiment
0,0,One of the other reviewers has mentioned that ...,1
1,1,A wonderful little production. <br /><br />The...,1
2,2,I thought this was a wonderful way to spend ti...,1
3,3,Basically there's a family where a little boy ...,0
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
sentiment_score = []
for idx,exmp in df_sentiment.iterrows():
  if(exmp['sentiment']==1):
    sentiment_score.append([0,1])
  else:
    sentiment_score.append([1,0])

In [ ]:
df_sentiment['sentiment_score'] = sentiment_score

In [ ]:
df_sentiment.head()

,Unnamed: 0,review,sentiment,sentiment_score
0,0,One of the other reviewers has mentioned that ...,1,"[0, 1]"
1,1,A wonderful little production. <br /><br />The...,1,"[0, 1]"
2,2,I thought this was a wonderful way to spend ti...,1,"[0, 1]"
3,3,Basically there's a family where a little boy ...,0,"[1, 0]"
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[0, 1]"


https://pytorch.org/docs/stable/nn.html#loss-functions

A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.

# For most of the BERT Tasks we need input_ids,token_type_ids,attention_mask

# Input_ids

Let text : "A Titan RTX has 24GB of VRAM"

After tokenization : ['A', 'Titan', 'R', '##T', '##X', 'has', '24', '##GB', 'of', 'V', '##RA', '##M']

Tokens are converted to ids : [101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102]

Decoding back : [CLS] A Titan RTX has 24GB of VRAM [SEP]

# Attention_mask

To tell the attention layers to ignore the padding tokens we need to pass them an attention mask . The attention mask is a binary tensor indicating the position of the padded indices so that the model does not attend to them.



#Token_type_ids

Some models’ purpose is to do classification on pairs of sentences or question answering.


text : [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]

token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

https://huggingface.co/docs/transformers/v4.16.2/en/glossary

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.review
        self.label = dataframe.sentiment_score
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            pad_to_max_length = True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'label': torch.tensor(self.label[index], dtype=torch.float),
            'text' : text
        }

1.   Design model (input,output size,forward pass)
2.   Construct loss and optimizer
3.   Training loop
    *   Forward pass : compute prediction
    *   backward pass : gradients
    *   update weights















In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_dataloader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        ids = ids.squeeze()
        mask = data['mask'].to(device, dtype = torch.long)
        mask = mask.squeeze()
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['label'].to(device, dtype = torch.float)
        outputs = model(ids, mask,token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss.backward()
        optimizer.step()
        scheduler.step()

In [ ]:
def validation(epoch):
    model.eval()
    final_targets=[]
    final_outputs=[]
    val_texts = []
    with torch.no_grad():
        for _, data in enumerate(valid_dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            ids = ids.squeeze()
            mask = data['mask'].to(device, dtype = torch.long)
            mask = mask.squeeze()
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['label'].to(device, dtype = torch.float)
            texts = data['text']

            outputs = model(ids, mask,token_type_ids)
            final_targets.extend(targets.cpu().detach().numpy().tolist())
            final_outputs.extend(outputs.cpu().detach().numpy().tolist())
            val_texts.extend(texts)
    return final_outputs, final_targets, val_texts

In [ ]:
train_df= df_sentiment.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(int(len(x)*0.8)))
val_df= df_sentiment.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(val_df.shape))

TRAIN Dataset: (4800, 4)
TEST Dataset: (1200, 4)


In [ ]:
?torch.utils.data.DataLoader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, max_length)
valid_dataset = CustomDataset(val_df, tokenizer, max_length)
train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': batch_size,
                'shuffle': False,
                'num_workers': 0
                }

train_dataloader = DataLoader(train_dataset, **train_params)
valid_dataloader = DataLoader(valid_dataset, **test_params)

In [ ]:
def loss_fn(outputs, targets):
    crossEntropy = torch.nn.CrossEntropyLoss()
    return crossEntropy(outputs, torch.argmax(targets,dim=1))

In [ ]:
optimizer = torch.optim.AdamW(params =  model.parameters(), lr = 2e-5, eps = 1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 200, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
for epoch in range(epochs):
    train(epoch)

In [ ]:
outputs, targets, texts = validation(epoch)


In [ ]:
import numpy as np
outputs = np.argmax(outputs, axis = 1)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['positive','negative']
y_true = np.argmax(targets, axis = 1)
y_pred = outputs
print(classification_report(y_true, y_pred,target_names=target_names))

In [ ]:
torch.save(model.state_dict(), "model_sentiment.bin")

In [7]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "We use Bayes' theorem to calculate the conditional probability."
encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
encoded_input.input_ids

tensor([[  101,  2057,  2224,  3016,  2229,  1005,  9872,  2000, 18422,  1996,
         18462,  9723,  1012,   102]])

In [9]:
tokenizer.tokenize(text)

['we',
 'use',
 'bay',
 '##es',
 "'",
 'theorem',
 'to',
 'calculate',
 'the',
 'conditional',
 'probability',
 '.']